In [ ]:
import pdfplumber
import pandas as pd
import re
import os

# for visa and amex

# Path to the folder containing PDFs
pdf_folder = "/Users/loschichis/Library/CloudStorage/OneDrive-Personal/Max/05 Projects/Financial Control/raw_pdfs"

# List to store extracted data
data = []

# List to store unprocessed lines
unprocessed = []

# Process each PDF in the folder
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        print(f"Processing: {pdf_file}")

        # Open the PDF
        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                text = page.extract_text()
                if text:  # Check if text was extracted
                    print(f"Page {page_number} of file {pdf_file} processed.")
                    # Split the text into lines
                    for line in text.split("\n"):
                        # Pattern to extract date, description, additional code, and amount
                        match = re.match(r"(\d{2}\.\d{2}\.\d{2})\s+\S+\s+(.+?)\s+(\d{8,})?\s*([\d.,]+)$", line)
                        if match:
                            fecha, descripcion, codigo_adicional, monto = match.groups()
                            # Replace commas with dots in the amount
                            monto = float(monto.replace(".", "").replace(",", "."))
                            data.append({
                                "Date": fecha,
                                "Description": descripcion,
                                "Additional Code": codigo_adicional if codigo_adicional else "N/A",
                                "Amount": monto
                            })
                        else:
                            unprocessed.append(line)  # Store unprocessed lines

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to a CSV file
output_path = "/Users/loschichis/Library/CloudStorage/OneDrive-Personal/Max/05 Projects/Financial Control/processed_data/extracted_data.csv"
df.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

# Save unprocessed lines to a log file
unprocessed_path = "/Users/loschichis/Library/CloudStorage/OneDrive-Personal/Max/05 Projects/Financial Control/processed_data/unprocessed_lines.txt"
with open(unprocessed_path, "w") as f:
    for line in unprocessed:
        f.write(line + "\n")
print(f"Unprocessed lines saved to {unprocessed_path}")


In [ ]:
# for mc

# Path to the folder containing PDFs
pdf_folder = "/Users/loschichis/Library/CloudStorage/OneDrive-Personal/Max/05 Projects/Financial Control/mc"

# List to store extracted data
data = []

# List to store unprocessed lines
unprocessed = []

# Process each PDF in the folder
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        print(f"Processing: {pdf_file}")

        # Open the PDF
        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                text = page.extract_text()
                if text:  # Check if text was extracted
                    print(f"Page {page_number} of file {pdf_file} processed.")
                    # Split the text into lines
                    for line in text.split("\n"):
                        # Pattern to extract date, description, code, and amount
                        match = re.match(r"(\d{2}-\w{3}-\d{2})\s+(.+?)\s+(\d+)?\s+([\d.,]+)$", line)
                        if match:
                            fecha, descripcion, codigo, monto = match.groups()
                            # Replace commas with dots in the amount
                            monto = float(monto.replace(".", "").replace(",", "."))
                            data.append({
                                "Date": fecha,
                                "Description": descripcion.strip(),
                                "Code": codigo if codigo else "N/A",
                                "Amount": monto
                            })
                        else:
                            unprocessed.append(line)  # Store unprocessed lines

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to a CSV file
output_path = "/Users/loschichis/Library/CloudStorage/OneDrive-Personal/Max/05 Projects/Financial Control/processed_data/extracted_data_mc.csv"
df.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

# Save unprocessed lines to a log file
unprocessed_path = "/Users/loschichis/Library/CloudStorage/OneDrive-Personal/Max/05 Projects/Financial Control/processed_data/unprocessed_lines_mc.txt"
with open(unprocessed_path, "w") as f:
    for line in unprocessed:
        f.write(line + "\n")
print(f"Unprocessed lines saved to {unprocessed_path}")
